In [7]:
import pandas as pd
from neo4j import GraphDatabase
from neo4j.exceptions import DriverError, Neo4jError

In [6]:
df = pd.read_csv('SP_500_ESG_Risk_Ratings.csv')
df.dropna(inplace=True)
df.drop(['ESG Risk Percentile', 'Description'], inplace=True, axis=1)
df

,Symbol,Name,Address,Sector,Industry,Full Time Employees,Total ESG Risk score,Environment Risk Score,Governance Risk Score,Social Risk Score,Controversy Level,Controversy Score,ESG Risk Level
0,A,Agilent Technologies Inc,"5301 Stevens Creek Boulevard\nSanta Clara, CA ...",Healthcare,Diagnostics & Research,"18,000",15.0,0.3,6.3,8.6,Low,1.0,Low
2,AAP,Advance Auto Parts Inc,"4200 Six Forks Road\nRaleigh, NC 27609\nUnited...",Consumer Cyclical,Specialty Retail,"40,000",12.0,0.0,3.0,8.0,Moderate,2.0,Negligible
3,AAPL,Apple Inc,"One Apple Park Way\nCupertino, CA 95014\nUnite...",Technology,Consumer Electronics,"164,000",17.0,0.6,9.2,6.9,Significant,3.0,Low
4,ABBV,Abbvie Inc,"1 North Waukegan Road\nNorth Chicago, IL 60064...",Healthcare,Drug Manufacturers—General,"50,000",28.0,1.1,9.9,16.8,Significant,3.0,Medium
5,ABC,Amerisourcebergen Corp,"1 West First Avenue\nConshohocken, PA 19428-18...",Healthcare,Medical Distribution,"46,000",12.0,1.3,5.2,5.6,Significant,3.0,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,XEL,Xcel Energy Inc,"414 Nicollet Mall\nMinneapolis, MN 55401\nUnit...",Utilities,Utilities—Regulated Electric,"11,982",24.0,12.9,4.2,6.8,Moderate,2.0,Medium
496,XRAY,Dentsply Sirona Inc,"13320 Ballantyne Corporate Place\nCharlotte, N...",Healthcare,Medical Instruments & Supplies,"15,000",16.0,2.0,7.0,7.0,Low,1.0,Negligible
497,XYL,Xylem Inc,"300 Water Street SE\nWashington, DC 20003\nUni...",Industrials,Specialty Industrial Machinery,"22,000",16.0,4.0,5.0,7.0,Low,1.0,Negligible
498,YUM,Yum Brands Inc,"1441 Gardiner Lane\nLouisville, KY 40213\nUnit...",Consumer Cyclical,Restaurants,"36,000",21.0,4.5,4.4,11.6,Moderate,2.0,Medium


In [29]:
class CompaniesFiller:
    def __init__(self, uri, user, password, database = None):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.database = database
    
    def create_companies(self, companies_df: pd.DataFrame):
        with self.driver.session() as session:
            for row in companies_df.iterrows():
                result = self._create_and_return_company(row[1])
                print(f'Created company: {result}')
            
    
    def _create_and_return_company(self, company_data):
        query = (
            "CREATE (cmp:Company { name: $company_name }) "
            "CREATE (sym:Symbol { name: $symbol }) "
            "CREATE (cmp)-[:SYMBOL]->(sym) "
            "CREATE (adr:Address { name: $address }) "
            "CREATE (cmp)-[:ADDRESS]->(adr) "
            "CREATE (sct:Sector { name: $sector }) "
            "CREATE (cmp)-[:SECTOR]->(sct) "
            "CREATE (ind:Industry { name: $industry }) "
            "CREATE (cmp)-[:INDUSTRY]->(ind) "
            "CREATE (emp:Number {value: $emp_amount}) "
            "CREATE (cmp)-[:EMPLOYEE_AMOUNT]->(emp) "
            "CREATE (esg_r_score:Number { value: $risk_score }) "            
            "CREATE (cmp)-[:RISK_SCORE]->(esg_r_score) "
            "CREATE (ct_level:Controversy_Level { value: $controversy_level }) "
            "CREATE (cmp)-[:CONTROVERSY_LEVEL]->(ct_level) "
            "CREATE (ct_score:Number { value: $controversy_score }) "
            "CREATE (cmp)-[:CONTROVERSY_SCORE]->(ct_score) "
            "RETURN cmp.name"
        )
        try:
            record = self.driver.execute_query(
                query_=query,
                company_name=company_data['Name'],
                symbol=company_data['Symbol'],
                address=company_data['Address'],
                sector=company_data['Sector'],
                industry=company_data['Industry'],
                emp_amount=int(company_data['Full Time Employees'].replace(',', '')),
                risk_score=company_data['Total ESG Risk score'],
                controversy_level=company_data['Controversy Level'],
                controversy_score=company_data['Controversy Score'],
                database_=self.database,
                result_transformer_=lambda r: r.single(strict=True)                
            )
            return record['cmp.name']
        except (DriverError, Neo4jError) as err:
            print(f'Error during query: {err}')
    
    def close(self):
        self.driver.close()

In [30]:
URI = 'bolt://localhost:7687'
USER = 'neo4j'
PASSWORD = 'ilikepkb'
DATABASE = 'companies'

db_filler = CompaniesFiller(URI, USER, PASSWORD, DATABASE)

In [31]:
try:
    db_filler.create_companies(df)
finally:
    db_filler.close()

Created company: Agilent Technologies Inc
Created company: Advance Auto Parts Inc
Created company: Apple Inc
Created company: Abbvie Inc
Created company: Amerisourcebergen Corp
Created company: Abbott Laboratories
Created company: Accenture Plc Cl A
Created company: Adobe Inc
Created company: Archer Daniels Midland Co
Created company: Automatic Data Processing
Created company: Ameren Corporation
Created company: Aes Corp
Created company: Aflac Inc
Created company: Arthur J Gallagher & Co
Created company: Akamai Technologies Inc
Created company: Allstate Corp
Created company: Applied Materials Inc
Created company: Ametek Inc
Created company: Amgen Inc
Created company: Ameriprise Financial Inc
Created company: American Tower Corp
Created company: Amazon.com Inc
Created company: Arista Networks Inc
Created company: Apa Corp
Created company: Air Products & Chemicals Inc
Created company: Amphenol Corp Cl A
Created company: Aptiv Plc
Created company: Atmos Energy Corp
Created company: Activi